In [2]:
% pylab
from astropy import constants
from scipy import integrate

G_val = constants.G.si.value

Cp_rock = 1000. # J/kg/K
M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3 # Mars, m
rho_mars = 3933 #kg/m^3
V_phobos = 5689*1e9 # +/- 60*1e9 m^3
M_phobos = 1.065*(10**16.) # Phobos, +/- .015 kg
R_phobos = 11.3*1e3  # Phobos, m
rho_phobos = 1860. # +/- 31 kg/m^3
ecc_phobos =  0.01511
a_phobos = 9378.*1e3 # Semi-major Axis, m 
w_phobos = 1./27562. # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [29]:
part_size = 1. # in m
y_local =  .35
y_nonlocal = 1.
a_r_initial = 1.6 # Initial centre location of the disk
Ring_width_int_fac  = .02*a_r_initial # initial half width = 0.02*a_r_initial (in R_Mars)


def func_visc(a_r_part,part_size,Surf_dens) :
    sigma_part = pi*part_size**2.
    M_part = (4.*pi/3.)*(part_size**3.)*(3000.) # mass of each particle 
    N_colmn_dens = Surf_dens/M_part
    Opt_depth = N_colmn_dens*sigma_part                 # tau
    
    ## Version with values across the ring ....
    n_mean = sqrt(G_val*M_mars/(a_r_part*R_mars)**3.)
    r_hill = a_r_part*R_mars*(2.*M_part/3./M_mars)**(1./3.)
    r_h_star = r_hill/2./part_size
    
    vel_disp1 = 2.*part_size*n_mean
    vel_disp2 = sqrt(2.*G_val*M_part/part_size)
    
    vel_disp_final = max(vel_disp1,vel_disp2)
    if Surf_dens < 0.001 :
        return 0.
    Q_value = (n_mean/G_val/3.36)*vel_disp_final/Surf_dens
    v_trans = y_local*(vel_disp_final**2.)*Opt_depth/(1. + Opt_depth**2.)/n_mean
    v_gravity = 0.
    v_nonlocal = y_nonlocal*(part_size**2.)*Opt_depth*n_mean
    v_total = v_trans + v_nonlocal + v_gravity
    #print(Q_value)
    if Q_value < 2. :
        vel_disp_final = Surf_dens*3.36*G_val/n_mean
        Q_value = 2.
        C_wake_term = min(53.*(r_h_star**5.),30.)
        v_trans    = .5*C_wake_term*(G_val*Surf_dens)**2./n_mean**3.
        v_gravity  = .5*C_wake_term*(G_val*Surf_dens)**2./n_mean**3.
        v_nonlocal = y_nonlocal*(part_size**2.)*Opt_depth*n_mean
        v_total  = v_trans + v_nonlocal  + v_gravity 

    return v_total

# Density evolution Equation ... - Init Setup
num_r_points = 400
r_grid = linspace(.9,3.5,num_r_points) # grid of r from .9 to 3. R_mars
dr_val = r_grid[1] - r_grid[0]

################# Section on initial conditions ... 
def gaussian_funcD(x, mu, sig): # mu - mean, sig - sigma
    norm = sqrt(2.*pi)*sig
    return 2.*pi*x*R_mars*exp(-(x - mu)**2./2./sig**2.)/norm # Integrand function for the total mass (x - r)

mass_total,err = integrate.quad(gaussian_funcD,min(r_grid),max(r_grid),args=(a_r_initial,Ring_width_int_fac),epsabs=1e-15)
scale_fact_dens = M_phobos/mass_total/R_mars  # Fix initial disk mass = Phobos Mass

def gaussian_func(x, mu, sig): # mu - mean, sig - sigma ... (Normalized so total Mass is )
    norm = sqrt(2.*pi)*sig
    return exp(-(x - mu)**2./2./sig**2.)/norm

dens_init = scale_fact_dens*gaussian_func(r_grid,a_r_initial , Ring_width_int_fac) # unit is kg/m^3
dens_init[dens_init < 0.001] = 0.
####################################################################

mars_atm_top_bin = argmin(abs(r_grid - (1.+ 2.*160*1e3/R_mars) ))
mars_atm_top_binB = argmin(abs(r_grid - (1.+30*1e3/R_mars) ))
tmp1_val = gaussian_func(r_grid[mars_atm_top_binB:mars_atm_top_bin-1],r_grid[mars_atm_top_binB],(-r_grid[mars_atm_top_binB] + r_grid[mars_atm_top_bin])/7.)
top_atm_visc = .1*tmp1_val/max(tmp1_val)

tmp2_val = gaussian_func(r_grid[0:mars_atm_top_binB],r_grid[mars_atm_top_binB],(r_grid[mars_atm_top_binB] - r_grid[1])/3.)
top_atm_visc2 = .1*tmp2_val/max(tmp2_val)*0.0 + .1
#r_grid[mars_atm_top_binB:mars_atm_top_bin-1],top_atm_visc2
#top_atm_visc

In [30]:
from scipy import interpolate
def visc_func(r_grid,part_size,dens_init,num_r_points):
    init_viscosity = r_grid.copy()*0.0
    for kp in range(0,num_r_points):
        init_viscosity[kp] = func_visc(r_grid[kp],part_size,dens_init[kp])
    #print(init_viscosity[mars_atm_top_bin-1:mars_atm_top_bin+5])
    init_viscosity[mars_atm_top_binB:mars_atm_top_bin-1] = init_viscosity[mars_atm_top_binB:mars_atm_top_bin-1] + top_atm_visc 
    ### Note that this is to a viscosity boundary 
    init_viscosity[0:mars_atm_top_binB] = top_atm_visc2     #init_viscosity[mars_atm_top_binB+1]+top_atm_visc2 
    return init_viscosity

def compute_flux(init_viscosity,dens_init,r_grid,dr_val):
    ## Next Step, Calculate the evolution ... 
    Flux_term_integrand = init_viscosity*dens_init*sqrt(r_grid*R_mars) 
    Flux_term_grad = gradient(Flux_term_integrand,dr_val*R_mars)  
    # Calculates - d/dr (v*sigma*sqrt(r)) - Goes left to right
    Flux_term_total = Flux_term_grad*sqrt(r_grid*R_mars)*6.*pi
    
    # Note that in detail, we want a staggered grid for the fluxes into/out of the bin -
    # So lets define a local interpolation function for flux(r)
    
    f_flux_r_term = interpolate.interp1d(r_grid, Flux_term_total,fill_value=0.,bounds_error=False) 
    new_r_grid1 = r_grid + dr_val/2.
    new_r_grid2 = r_grid - dr_val/2.
    #print(new_r_grid1,f_flux_r_term)
    flux_dsigma = f_flux_r_term(new_r_grid1) - f_flux_r_term(new_r_grid2)
    # Note, be careful to use it only for something outside the range of r_grid (set that to be 0.)
    Bin_area_func = 2.*pi*R_mars*r_grid*dr_val*R_mars
    flux_dsigma[r_grid <.95] =  0. # No flux at the bndry condition ... 
    flux_dsigma[r_grid > 1.95] = 0. # No flux at the bndry condition ... 
    return flux_dsigma/Bin_area_func

In [24]:
import time
steps_to_do = 800000
dt_init = 8e4*3.154e7       # unit is per seconds
#max_t = 80.*1e6*3.154e7                            # unit is per seconds

dens_all_time = zeros([num_r_points,steps_to_do/10+1])
t_now = zeros([steps_to_do,1])
dt = 8e4*3.154e7

dens_temp_var = dens_init.copy()
t_now_temp = 0.
dens_all_time[:,0] = dens_init.copy()
mass_all_times = zeros([steps_to_do,1])
t_all_times = zeros([steps_to_do,1])

for i in range(0,steps_to_do-1):
        #t1 = time.time()
        visc_all = visc_func(r_grid,part_size,dens_temp_var,num_r_points)
        visc_all2 = visc_all.copy()
        visc_all2[visc_all2<=0] = 1e-8 
        max_dt_allowed = min(((dr_val*R_mars)**2.)/visc_all2)/dt
        dt = min(.5*max_dt_allowed*dt,dt_init)
        flux_term_r = compute_flux(visc_all,dens_temp_var,r_grid,dr_val)
        dens_temp_var += flux_term_r*dt
        mass_all_times[i] = sum(dens_temp_var*2.*pi*dr_val*R_mars*r_grid*R_mars)
        #dens_temp_var[r_grid<1.] = 0.
        t_now_temp +=dt
        t_all_times[i] = t_now_temp 
        if (steps_to_do%10.) == 0. :
            dens_all_time[:,i/10+1] = dens_temp_var
            t_now[i/10+1] = t_now_temp # in Seconds 
        
        
        
print("Final_Time(Myr) :",(t_now_temp/3.154e7/1e6))
#savez('output_1p6_run',t_now[0:950]/3.154e7/1e6,dens_all_time[:,0:950],r_grid)
#npzfile = np.load('output_1p6_run.npz')

Final_Time(Myr) : 0.445192923381


In [28]:
figure(3)
clf()
for k in range(15,160): #int(steps_to_do/100+1)):
    plot(r_grid,dens_all_time[:,k*500])

#plot(r_grid,50. + r_grid*0.0,'k--')
#t_now[1000]/3.154e7/1e6

In [7]:
figure(3)
clf()
plot(t_all_times/3.154e7/1e6,mass_all_times/M_phobos,'ko')
grid('on')

In [8]:
npzfile = np.load('output_1p6_run.npz')
dens_all_time = npzfile['arr_1']

In [23]:
npzfile.files

['arr_0', 'arr_1', 'arr_2']

In [60]:
min((dr_val*R_mars)**2./visc_all)/dt

22.827387514297524

In [22]:
shape(dens_all_time)

(200, 20001)

In [ ]:
## Viscosity  atmosphere - Mars
h = linspace(10,160e3,1000)
rho_atmp = 1.76e-2*exp(-h/11./1e3)
mu = (4.6e-6 + (1.5e-5 - 4.6e-6)*h/160e3)/rho_atmp

semilogy(h,mu)